### Compare variant calls and annotations from two RNA-seq experiments

In [1]:
import sys
import pandas as pd
import os

#input_file1 = "/cluster/db/mecoulter/CEMartinez_RNAseq-209915706/results/bcftools/3H_locus_filtered.anno.vcf"
input_file2 = "/cluster/db/mecoulter/RNAseq2/bcftools/3H_locus_filtered.anno.vcf"

first_set = set()
second_set = set()

locus = [33181340, 36970860]#3H locus boundaries on 3H
"""
for line in open(input_file1):#Barke - Int19/Int56
    if line.startswith("#"):
        continue
    fields = line.rstrip().split("\t")
    SNP_position = fields[1]
    ref = fields[3]
    alt = fields[4]
    variant_id = f"{SNP_position}_{ref}_{alt}"
    first_set.add(variant_id)

print(f"{len(first_set)} unique variants in experiment one")
"""
for line in open(input_file2):#Barke/Int_124_17 - Int_124_52
    if line.startswith("#"):
        continue
    fields = line.rstrip().split("\t")
    SNP_position = fields[1]
    if int(SNP_position) < locus[0] or int(SNP_position) > locus[1]:
        continue
    ref = fields[3]
    alt = fields[4]
    variant_id = f"{SNP_position}_{ref}_{alt}"
    second_set.add(variant_id)

print(f"{len(second_set)} unique variants in experiment two")

545 unique variants in experiment two


* What variants are shared between two experiments?

In [ ]:
#print(f"Number of shared variants: {len(first_set & second_set)}, proportion (compared to first set): {len(first_set & second_set)/len(first_set)} ")

In [4]:
input_file = input_file2

annotation_input = "/cluster/db/mecoulter/BaRT2v18/BaRT_2_18_annotation_genes.txt"




class SnpEffAnnotation:
    def __init__(self, annotation):
        try:
            self.allele, self.p_annotation, self.putative_impact, self.position, self.gene, self.feature_type, self.transcript, self.transcript_type  = annotation.split("|")[:8]
        except ValueError:
            print(f"Error at position {SNP_position}! Annotation is {annotation}")
            sys.exit()
    def __repr__(self):
        return self.p_annotation

class SnpEff:
    gene_impact = {}
    gene_SNP_annotation = {}
    Instances = {}#Gene: {position1: [annotations], position2: [annotations]}
    Instances_clean = {}
    position_info = {}
    dn_ds = {}
    
    def __init__(self, line):
        self.fields = line.rstrip().split("\t")
        self.INFO = self.fields[7]
        self.SNP_position = self.fields[1]
        self.reference_allele = self.fields[3]
        self.alternative_allele = self.fields[4]
        annotations = self.INFO.split("ANN=")[1].split("=")[0].split(",")#List of all possible annotations
        self.annotations = []
        for a in annotations:
            annotation = SnpEffAnnotation(a)
            self.annotations.append(annotation)
            self.gene_impact.setdefault(annotation.gene,[]).append(annotation.putative_impact)  
            
            if annotation.putative_impact == "HIGH" or annotation.putative_impact == "MODERATE":
                self.gene_SNP_annotation.setdefault(annotation.gene,[]).append(f"{annotation.p_annotation} : {self.SNP_position}")
                
            try:
                position_annotations = self.Instances[annotation.gene]
                position_annotations.setdefault(self.SNP_position, []).append(annotation)
                self.Instances[annotation.gene] = position_annotations
            except KeyError:
                positions_annotations = {}
                positions_annotations[self.SNP_position] = [annotation]
                self.Instances[annotation.gene] = positions_annotations
                
        self.position_info[self.SNP_position] = self
                
    def __getitem__(self, item):
        if self.Instances_clean:
            return self.Instances_clean[item]
        else:
            return self.Instances[item]
    
    def clean_up_annotations(self):
        """Many positions have multiple annotations.
        This is because of multiple transcripts. Most transcripts will be similar, 
        so will effectively have the same annotation. So remove duplicates. Will
        also remove annotations not of interest e.g upstream/ downstream"""
        
        not_interesting = {"downstream_gene_variant", "upstream_gene_variant", \
        "intergenic_region", "intron_variant"}
        
        for gene, position_annotation in self.Instances.items():
            position_annotation_clean = {}
            for position, annotations in position_annotation.items():
                annotations_cleaned = []
                all_p_annotations = set()
                for annotation in annotations:
                    if annotation.p_annotation not in all_p_annotations and annotation.p_annotation not in not_interesting:
                        #if annotation.putative_impact == "HIGH" or annotation.putative_impact == "MODERATE":
                        annotations_cleaned.append(annotation)
                        all_p_annotations.add(annotation.p_annotation)
                if annotations_cleaned:#Only keep high/moderate impact snps
                    position_annotation_clean[position] = annotations_cleaned
            self.Instances_clean[gene] = position_annotation_clean
    def __repr__(self):
        string = ""
        print_dict = self.Instances_clean if self.Instances_clean else self.Instances
        
        for gene, position_annotation in print_dict.items():
            string += f"{gene}: \n"
            for position, annotations in position_annotation.items():
                string += f"{position} : {','.join([str(annotation) for annotation in annotations])}\n"
            string += "\n"  
        return string
    
    def __len__(self):
        return(len(self.Instances.keys()))
    
    def write_tsv(self, outfile, gene_annotation):
        """Write a .csv file with results of cleaned up annotation. 
        Gene: Position: Annotation"""
        with open(outfile, "w") as out:
            out.write("Gene\tPosition\tAnnotation\tref allele\talt allele\tPanzzer Annotation\tdn/ds ratio\n")
            for gene, position_annotation in self.Instances_clean.items():
                
                index = 0
                
                try:
                    panz_annotation = gene_annotation.at[gene,"Pannzer annotation"]
                except KeyError:
                    print(f"KeyError for gene {gene}")
                    panz_annotation = "Not found"
                    
                try:
                    dn_ds = self.dn_ds[gene]
                except KeyError:
                    dn_ds = ""
                    
                for position, annotations in position_annotation.items():
                    
                    ref = self.position_info[position].reference_allele
                    alt = self.position_info[position].alternative_allele
                    
                    out.write(f"{gene}\t") if not index else out.write(f"\t")
                    
                    out.write(f"{position}\t{','.join([str(annotation) for annotation in annotations])}\t\
                    {ref}\t{alt}\t")#
                    
                    #Only write annotation once
                    out.write(f"{panz_annotation}\t{dn_ds}\n") if not index else out.write("\t\n")
                    
                    index += 1
            
    def get_dn_ds(self):
        
        """Calculates dn/ds ratio for each gene where possible
        missense_variant = n, synonymous_variant"""
        for gene, position_annotation in self.Instances_clean.items():
            n = []
            s = []
            for position, annotations in position_annotation.items():
                SNP_anotations = set([str(annotation) for annotation in annotations])
                if "missense_variant" in SNP_anotations and "synonymous_variant" in SNP_anotations:
                    continue#A case where SNP is annotated as both, these cannot be used
                n += [annotation for annotation in SNP_anotations if annotation == "missense_variant"]
                s += [annotation for annotation in SNP_anotations if annotation == "synonymous_variant"]
            if n and s:
                self.dn_ds[gene] = len(n)/len(s)
            else:
                self.dn_ds[gene] = "NA"
            
                
                    
                
            
        
        
        
            


gene_annotation = pd.read_csv(annotation_input,engine="c",index_col=0, sep="\t")

#expressed_genes = set(gene_annotation.index)



impact = {"MODIFIER":0, "LOW": 0, "MODERATE": 0, "HIGH": 0 }
locus = [33181340, 36970860]#3H locus boundaries on 3H

for line in open(input_file):
    #print(line)
    if line.startswith("#"):
        continue
    #print(line)
    position = int(line.rstrip().split("\t")[1])
    if position < locus[0] or position > locus[1]:#ONly include snps in 3H locus
        continue
    snpeff = SnpEff(line)

snpeff.clean_up_annotations()

#print(snpeff)
#print(f"Length of Instances is {len(snpeff)}")

snpeff.get_dn_ds()#Get dn/ds ratio for each gene

snpeff.write_tsv("/cluster/db/mecoulter/RNAseq2/3H_locus_snpeff_gene_annotation.tab", gene_annotation)


KeyError for gene BaRT2v18chr3HG122940-BaRT2v18chr3HG122950
KeyError for gene BaRT2v18chr3HG122950-BaRT2v18chr3HG122960
KeyError for gene BaRT2v18chr3HG122980-BaRT2v18chr3HG122990
KeyError for gene BaRT2v18chr3HG122990-BaRT2v18chr3HG123000
KeyError for gene BaRT2v18chr3HG123000-BaRT2v18chr3HG123010
KeyError for gene BaRT2v18chr3HG123010-BaRT2v18chr3HG123030
KeyError for gene BaRT2v18chr3HG123030-BaRT2v18chr3HG123040
KeyError for gene BaRT2v18chr3HG123040-BaRT2v18chr3HG123060
KeyError for gene BaRT2v18chr3HG123110-BaRT2v18chr3HG123120
KeyError for gene BaRT2v18chr3HG123150-BaRT2v18chr3HG123170
KeyError for gene BaRT2v18chr3HG123180-BaRT2v18chr3HG123200
KeyError for gene BaRT2v18chr3HG123210-BaRT2v18chr3HG123220
KeyError for gene BaRT2v18chr3HG123390-BaRT2v18chr3HG123400
KeyError for gene BaRT2v18chr3HG123410-BaRT2v18chr3HG123420
KeyError for gene BaRT2v18chr3HG123420-BaRT2v18chr3HG123430
KeyError for gene BaRT2v18chr3HG123440-BaRT2v18chr3HG123450
KeyError for gene BaRT2v18chr3HG123490-B

In [8]:
expression_info = "/cluster/db/mecoulter/RNAseq2/RNAquant_analysis_result/all_3H_genes_expressed.csv"

snp_annotation = pd.read_csv("/cluster/db/mecoulter/RNAseq2/3H_locus_snpeff_gene_annotation.tab",\
 engine="c", sep="\t")

expression = pd.read_csv(expression_info, engine="c", index_col=1)

expressed_genes = set(expression.index)
#snp_annotation.head()

snp_annotation_expressed = snp_annotation

drop = False
for i in snp_annotation.index:
    if str(snp_annotation.at[i,"Gene"]) != "nan":
        if snp_annotation.at[i,"Gene"] not in expressed_genes:
            snp_annotation_expressed = snp_annotation_expressed.drop([i])
            drop = True
        else:
            drop = False
    else:
        if drop:
            snp_annotation_expressed = snp_annotation_expressed.drop([i])

snp_annotation_expressed.to_csv("/cluster/db/mecoulter/RNAseq2/3H_locus_snpeff_gene_annotation_expressed.tab", sep="\t")
            


In [6]:
snp_annotation_expressed.head()

,Gene,Position,Annotation,ref allele,alt allele,Panzzer Annotation,dn/ds ratio
176,BaRT2v18chr3HG123110,34748456,5_prime_UTR_variant,G,C,NaN,5.0
177,NaN,34748574,5_prime_UTR_variant,G,T,NaN,NaN
178,NaN,34748648,5_prime_UTR_variant,A,AG,NaN,NaN
179,NaN,34749036,5_prime_UTR_variant,C,G,NaN,NaN
180,NaN,34749044,5_prime_UTR_premature_start_codon_gain_variant...,C,G,NaN,NaN


In [7]:
expression.head()

,Unnamed: 0,contrast,adj.pval,log2FC,up.down,Chromosome,Start,End,Strand,Pannzer annotation
target,,,,,,,,,,
BaRT2v18chr3HG150360,1,HEB_124_52-HEB_124_17,2.251004e-09,-10.652320,down-regulated,chr3H,496316055,496319769,-,NaN
BaRT2v18chr3HG152530,2,HEB_124_52-HEB_124_17,2.251004e-09,-11.909750,down-regulated,chr3H,512534450,512536358,+,NaN
BaRT2v18chr3HG120970,3,HEB_124_52-HEB_124_17,9.944333e-09,-5.106883,down-regulated,chr3H,22786096,22788049,-,NaN
BaRT2v18chr3HG121930,4,HEB_124_52-HEB_124_17,9.944333e-09,-10.847032,down-regulated,chr3H,26903372,26907471,-,NaN
BaRT2v18chr3HG122380,5,HEB_124_52-HEB_124_17,9.944333e-09,-9.825658,down-regulated,chr3H,29462547,29468929,-,PHB domain-containing protein


In [71]:
expressed_genes[0:5]

Index(['BaRT2v18chr1HG000020', 'BaRT2v18chr1HG000040', 'BaRT2v18chr1HG000050',
       'BaRT2v18chr1HG000060', 'BaRT2v18chr1HG000070'],
      dtype='object')